In [1]:
comment_prompt = """
你是一位专业的前端专家，擅长于代码的优化，你现在需要根据用户的输入，调整代码中的依赖顺序，然后返回调整后的代码，调整的规则如下：
1. 先引入第三方依赖，然后是包含别名的项目依赖，接着是不包含别名使用相对路径的项目依赖，最后是less文件的样式依赖。
2. 你只调整用户代码中依赖的顺序，不对用户提供的代码做出任何其它的修改。
3. 返回代码字符串，返回的代码不要包含任何Markdown格式标记。

- 调整示例:
  - 例子1：调整依赖引入顺序
    # 原始代码     
    import { Component } from 'react';
    import { Button } from 'antd';
    import styles from './styles.less';
    import { utils } from './utils';
    import { config } from '@alias/config';

    # 调整后的代码
    import { Component } from 'react';
    import { Button } from 'antd';
    import { config } from '@alias/config';
    import { utils } from 'utils';
    import styles from './styles.less';

  - 例子2：输出格式

    # 错误的输出格式1
    ```javascript
    import { Component } from 'react';
    ```

    # 正确的输出格式2
    import { Component } from 'react';

    # 错误的输出格式2
    ```javascript
    /**\n * 企业logo\n */
    ```

    # 正确的输出格式2
    /**
    * 企业logo
    */
"""


In [2]:
from huggingface_hub import InferenceClient
import os

api_key = os.environ["HUGGINGFACE_API_KEY"]
if not api_key:
    raise ValueError("HUGGINGFACE_API_KEY environment variable is not set.")

client = InferenceClient(model="Qwen/Qwen2.5-Coder-32B-Instruct", api_key=api_key)

# 根据传入的文本，调用大模型处理返回结果
def prompt_model(content):
  code = repr(content) 

  messages = [
      {"role": "system", "content": comment_prompt},
      {"role": "user", "content": code}
  ]

  try:
    response_content=client.chat_completion(
      messages=messages,
      max_tokens=5000,
      # response_format="text",
      temperature=0.0,
      # stream=True,
    )
    response_content = response_content.choices[0].message.content
    # print(response_content)
    return response_content
  except Exception as e:
      print(f"An error occurred: {e}")
   


In [3]:
import sys
module_path = '../../package'
if module_path not in sys.path:
    sys.path.append(module_path)
from jupyter_ui import open_dir


open_dir.open_dir_ui()

GridBox(children=(Button(button_style='success', description='选择解析目录', style=ButtonStyle(), tooltip='选择目录，目录下所…

2024-11-19 20:01:25.932 python[85877:1514087] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-19 20:01:27.150 python[85877:1514087] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


In [11]:
from pathlib import Path

text = open_dir.dir_text.value


# 设置起始目录
start_dir = Path(text)


def write_res(content, write_path):
    write_path.write_text(content)


def run(dir_path, is_write=True, exclude_dirs=[]):
    # 判断是否是目录
    if not dir_path.is_dir():
        print(f"The provided path is not a directory: {start_dir}")
        content = start_dir.read_text()
        res_content = prompt_model(content)
        write_path = start_dir if is_write else Path("../data/res.tsx")

        print(res_content)
        print(write_path)

        write_res(res_content, write_path)
        exit(1)

    # # 遍历目录
    for file_path in start_dir.rglob("*"):
        # 检查当前路径是否在需要排除的目录中
        print("parts",file_path.parts)
        if any(excluded in file_path.parts for excluded in exclude_dirs):
            continue
        print(file_path)
        # 检查文件后缀名
        if file_path.suffix in [".jsx", ".tsx"]:
            # 打印文件路径
            print(f"Found file: {file_path}")
            # 读取文件内容
            try:
                content = file_path.read_text()
                res_content = prompt_model(content)
                # 生成结果写入文件中
                write_path = file_path if is_write else Path("../data/res.tsx")
                write_res(res_content, write_path)
                print("------")  # 分隔线，用于区分不同文件的内容
            except Exception as e:
                print(f"Error reading file {file_path}: {e}")



run(start_dir, is_write=True, exclude_dirs=["business"])



parts ('/', 'Users', 'liepin', 'liepin-project', 'fe-mskh-pc', 'v6', 'src', 'components', 'JobTags')
/Users/liepin/liepin-project/fe-mskh-pc/v6/src/components/JobTags
parts ('/', 'Users', 'liepin', 'liepin-project', 'fe-mskh-pc', 'v6', 'src', 'components', 'DomainChangeResult')
/Users/liepin/liepin-project/fe-mskh-pc/v6/src/components/DomainChangeResult
parts ('/', 'Users', 'liepin', 'liepin-project', 'fe-mskh-pc', 'v6', 'src', 'components', 'NestEmpty')
/Users/liepin/liepin-project/fe-mskh-pc/v6/src/components/NestEmpty
parts ('/', 'Users', 'liepin', 'liepin-project', 'fe-mskh-pc', 'v6', 'src', 'components', 'CustomDrawer')
/Users/liepin/liepin-project/fe-mskh-pc/v6/src/components/CustomDrawer
parts ('/', 'Users', 'liepin', 'liepin-project', 'fe-mskh-pc', 'v6', 'src', 'components', 'FaceRecognition')
/Users/liepin/liepin-project/fe-mskh-pc/v6/src/components/FaceRecognition
parts ('/', 'Users', 'liepin', 'liepin-project', 'fe-mskh-pc', 'v6', 'src', 'components', 'CandidateTable')
/User